In [314]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [315]:
df = pd.read_csv("./data/lap/f1/2021/22.csv")

pivot = df.pivot("lap",  "driver_name", "cum_lap_time")
# Create deep copy TODO: create empty dataframe
overtaken = pivot.copy(deep=True)
overtaking = pivot.copy(deep=True)
clean_lap = pivot.copy(deep=True)
curr_lap = pivot.fillna(np.Infinity)
next_lap = pivot.shift(1).fillna(np.Infinity)
cols = list(overtaken.columns.copy(deep=True))
for driver in cols:
    overtaken[driver] = np.empty((len(pivot), 0)).tolist()
    overtaking[driver] = np.empty((len(pivot), 0)).tolist()
    clean_lap[driver] = True
    for opponent in cols:
        if driver != opponent:
            # Compute if one driver got overtaken during the lap
            overtaken_opponent_driver_cond = ( curr_lap[driver]   < curr_lap[opponent])     & (next_lap[driver]   > next_lap[opponent])
            overtaking_opponent_driver_cond = (curr_lap[opponent] < curr_lap[driver])       & (next_lap[opponent] > next_lap[driver])
            # Clean lap is approximation, if driver is less than 3 seconds behind opponent, then lap is not clean
            clean_lap_opponent_driver_cond = ( curr_lap[driver]   - curr_lap[opponent] > 3) | (curr_lap[driver] - curr_lap[opponent] < 0)

            overtaken .loc[overtaken_opponent_driver_cond, driver]  = overtaken .loc[overtaken_opponent_driver_cond, driver] .apply(lambda x: x + [opponent])
            overtaking.loc[overtaking_opponent_driver_cond, driver] = overtaking.loc[overtaking_opponent_driver_cond, driver].apply(lambda x: x + [opponent])
            clean_lap[driver] = clean_lap[driver] & clean_lap_opponent_driver_cond
# Join all dataframe
df_indexed = df.set_index(["driver_name", "lap"])
df_overtaken = df_indexed.join(overtaken.melt(ignore_index=False).rename({"value": "overtaking"}, axis=1).reset_index().set_index(["driver_name", "lap"]), how="left")
df_overtaking = df_overtaken.join(overtaking.melt(ignore_index=False).rename({"value": "overtaken_by"}, axis=1).reset_index().set_index(["driver_name", "lap"]), how="left")
df_cleanlap = df_overtaking.join(clean_lap.melt(ignore_index=False).rename({"value": "clean_lap"}, axis=1).reset_index().set_index(["driver_name", "lap"]), how="left").reset_index()

In [319]:
df_cleanlap.loc[(df_cleanlap["driver_name"] == "perez")]

,driver_name,lap,race_name,lap_time,position,cum_lap_time,overtaking,overtaken_by,clean_lap
2,perez,1,Abu Dhabi Grand Prix,93.756,3,93.756,[],[],False
21,perez,2,Abu Dhabi Grand Prix,89.507,3,183.263,[],[],False
40,perez,3,Abu Dhabi Grand Prix,89.273,3,272.536,[],[],False
59,perez,4,Abu Dhabi Grand Prix,88.902,3,361.438,[],[],False
78,perez,5,Abu Dhabi Grand Prix,89.249,3,450.687,[],[],False
97,perez,6,Abu Dhabi Grand Prix,89.239,3,539.926,[],[],False
116,perez,7,Abu Dhabi Grand Prix,89.022,3,628.948,[],[],True
135,perez,8,Abu Dhabi Grand Prix,88.940,3,717.888,[],[],True
154,perez,9,Abu Dhabi Grand Prix,89.063,3,806.951,[],[],True
173,perez,10,Abu Dhabi Grand Prix,88.937,3,895.888,[],[],True
